In [36]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import psycopg2 as ps

In [37]:
# Creamos la conexión a la base de datos
# Vamos a crear una conexión a la base de datos.
conn = ps.connect(
    dbname = "BBDD_Hoteles", # base a la que nos queremos conectar
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432" # puerto en el que s eencuentra postgres
)

In [38]:
# Creamos un cursor el cual nos va a permitir ejecutar querys.
cur = conn.cursor()

In [39]:
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

## Bonus Track 2

## Consulta 1: Cuantos hoteles tiene la base de datos


In [40]:
query1 = """
    SELECT 
        count(id_hotel) 
    FROM hoteles; 
"""
cur.execute(query1)
cur.fetchall()

[(29,)]

## Consulta 2: Cuantas reservas se han hecho

In [41]:
query2 = """
    SELECT 
        count(id_reserva) 
    FROM reservas; 
"""
cur.execute(query2)
cur.fetchall()

[(15000,)]

## Consulta 3: Identifica los 10 clientes que más se han gastado


In [42]:
query3 = """
    SELECT 
        c.nombre,
        sum(r.precio_noche)
    FROM clientes as c
        INNER JOIN  reservas as r ON c.id_cliente = r.id_cliente
    GROUP BY c.nombre
    ORDER BY 2 DESC
    LIMIT 10; 
"""
cur.execute(query3)
cur.fetchall()

[('Nazaret', 9194.233058437145),
 ('Reyes', 8830.594966228322),
 ('José', 7976.855523083232),
 ('Amor', 7440.725654803355),
 ('Trinidad', 7306.092914252774),
 ('Cruz', 7294.01557152846),
 ('Guiomar', 7170.887398393007),
 ('Amancio', 7145.372831655316),
 ('René', 6587.434547319466),
 ('Guadalupe', 6520.99880800067)]

## Consulta 4: Identifica el hotel de la competencia y el hotel de nuestra marca que más han recaudado para esas fechas


In [43]:
conn.rollback()

In [44]:
# Hotel competencia con mas recaudacion
query4 = """ 
        SELECT 
            h.nombre_hotel,
            sum(r.precio_noche)
        FROM hoteles as h
            INNER JOIN  reservas as r ON h.id_hotel = r.id_hotel
        WHERE h.competencia = True
        GROUP BY h.nombre_hotel
        ORDER BY 2 DESC
        LIMIT 1; 
"""

cur.execute(query4)
cur.fetchall()

[('ibis Madrid Centro las Ventas', 90128.0)]

In [45]:
# Hotel grupo con mas recaudacion
query4_1 = """ 
        SELECT 
            h.nombre_hotel,
            sum(r.precio_noche)
        FROM hoteles as h
            INNER JOIN  reservas as r ON h.id_hotel = r.id_hotel
        WHERE h.competencia = False
        GROUP BY h.nombre_hotel
        ORDER BY 2 DESC
        LIMIT 1; 
"""

cur.execute(query4_1)
cur.fetchall()

[('Hotel Monte Verde', 152136.8095238094)]

## Consulta 5: Identifica cuantos eventos hay.


In [46]:
# Hotel competencia con mas recaudacion
query5 = """ 
        SELECT 
            count(id_evento)
        FROM eventos ; 
"""

cur.execute(query5)
cur.fetchall()

[(217,)]

## Consulta 6: Identifica el día que más reservas se han hecho para nuestro hoteles

In [47]:
# Hotel competencia con mas recaudacion
query6 = """ 
        SELECT 
            r.fecha_reserva,
            count(r.id_reserva)
        FROM hoteles as h
            INNER JOIN  reservas as r ON h.id_hotel = r.id_hotel
        WHERE h.competencia = True
        GROUP BY r.fecha_reserva ; 
"""

cur.execute(query6)
cur.fetchall()

[(datetime.date(2025, 2, 21), 5172)]

In [48]:
# una vez hemos terminado de trabajar es necesario cerrar la conexión y el cursor.
cur.close()
conn.close()

## Bonus Track 2